In [1]:
!nvidia-smi

Tue Aug  4 16:44:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 16.2MB/s 
     |████████████████████████████████| 3.0MB 28.0MB/s 
     |████████████████████████████████| 890kB 35.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4b11fd54c833cf49cc7ffa2ac96d9ae7977ed8d9bfc17004dc5299a55f023387
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [15]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 3.4MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [30]:
!unzip -q "/content/gdrive/My Drive/Yelp_Sentiment_Analysis/dataset.zip" -d '/content'

**Prediction**

In [4]:
import torch
from transformers import BertTokenizer, BertModel

import re
import numpy as np

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# Key Tokens
print('[CLS] token: ', tokenizer.convert_tokens_to_ids("[CLS]"))
print('[SEP] token: ', tokenizer.convert_tokens_to_ids("[SEP]"))
print('[PAD] token: ', tokenizer.convert_tokens_to_ids("[PAD]"))

[CLS] token:  101
[SEP] token:  102
[PAD] token:  0


In [7]:
def clean_text(text):
    # basic text preprocessing
    text = text.replace("''", '" ').replace("``", '" ')  # replace the quotes 
    text = text.replace("`", "'") # backticks typo
    text = text.replace("\"", "") # replace quotes
    text = text.replace("...", " ").replace(". . .", " ").replace('..', ' ') # replace dots
    text = text.replace("\n", " ") # replace new line chars
    text = re.sub(r'(?:http:|https:).*?(?=\s)', '', text)  # remove url and website
    text = re.sub(r'www.*?(?=\s)', '', text)  # remove url and website

    list_to_replace = [':(', '=)', ':)', ':P', '-', ',,', ':', ';', '/', '+', '~', '_', '*', '(', ')', '&', '=', '@'] #replace the punctuations which are messy with empty
    for elem in list_to_replace:
        text = text.replace(elem, '')
    
    text = re.sub(r'\!{2,}', '!', text) # duplicate punctuation
    text = re.sub(r'\?{2,}', '?', text) # duplicate punctuation
    text = text.replace('?!', '?').replace('!?', '?') #replace slang punctuation with question
    text = re.sub(r'\s(?:\.|\,)', '', text) # replace spaces before punctuation
    text = re.sub(r'([a-zA-Z?!])\1\1+', r'\1', text) # removes repeated characters (Ex: Veryyyyy -> very)
    
    text = re.sub(r'\s{2,}', ' ', text) # replace multiple spaces
    text = text.strip() # strips spaces

    text = text.lower() # lower text

    return text

In [8]:
def pad_sent(raw_text, max_text_len = 350):  #token number 0 is [PAD]
    curr_text = "[CLS] " + raw_text  # add starting cls token
    tokenized_text = tokenizer.tokenize(curr_text) # tokenize            
    tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized_text) # convert to ids

    tokenized_ids = tokenized_ids[:max_text_len - 1]   # trim the reviews
    tokenized_ids.append(102) # add special token for [SEP]
    
    # get text length and padding
    curr_sent_len = len(tokenized_ids)
    remaining = max_text_len - curr_sent_len # words remaining for padding

    # pad the input token
    tokenized_ids.extend([0] * remaining)  # pad the text to max_text_len

    # create attention and segmented mask
    curr_attn = [1] * curr_sent_len  
    curr_attn.extend([0] * remaining)
    curr_seg_id = [0] * max_text_len

    return tokenized_ids, curr_attn, curr_seg_id

In [9]:
test_string = 'I hate this FOOD!!!'
test_string = clean_text(test_string)
tokenized_ids, curr_attn, curr_seg_id = pad_sent(test_string)

**Data Loader**

In [10]:
import torch
import torch.utils.data as data
import numpy as np

In [11]:
class YelpDataset(data.Dataset):
    """Yelp Dataset.

    Each item in the dataset is a tuple with the following entries (in order):
         text = np.array(text),
         attnmask = np.array(attnmask),
         seg_id = np.array(seg_id),
         rating = np.array(rating),
         ids = np.array(uniq_ids)

    Args:
        data_path (str): Path to .npz file containing pre-processed dataset.
    """
    def __init__(self, data_path):
        super(YelpDataset, self).__init__()

        dataset = np.load(data_path)
        self.text = torch.from_numpy(dataset['text']).long()
        self.attnmask = torch.from_numpy(dataset['attnmask']).long()
        self.seg_id = torch.from_numpy(dataset['seg_id']).long()
        self.rating = torch.from_numpy(dataset['rating']).long()
        self.ids = torch.from_numpy(dataset['ids']).long()

        # index
        self.valid_idxs = [idx for idx in range(len(self.ids))]

    def __getitem__(self, idx):
        idx = self.valid_idxs[idx]
        example = (self.text[idx],
                   self.attnmask[idx],
                   self.seg_id[idx],
                   self.rating[idx],
                   self.ids[idx])
        return example

    def __len__(self):
        return len(self.valid_idxs)

In [12]:
dev_dataset = YelpDataset('gdrive/My Drive/Yelp_Sentiment_Analysis/test.npz')
dev_loader = data.DataLoader(dev_dataset,
                                batch_size=16,
                                shuffle=False,
                                num_workers=4,
                                )

**BERT FINETUNE**

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.optim.lr_scheduler import StepLR
from transformers import BertTokenizer, BertModel, BertConfig

import numpy as np
import tqdm
#from ujson import load as json_load
from collections import OrderedDict
from json import dumps
import random
import os
import logging
import queue
import shutil
import string
import re
import json
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

random.seed(224)
np.random.seed(224)
torch.manual_seed(224)
torch.cuda.manual_seed_all(224)

from tensorboardX import SummaryWriter

In [17]:
class AverageMeter:
    """Keep track of average values over time.

    Adapted from:
        > https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    def __init__(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        """Reset meter."""
        self.__init__()

    def update(self, val, num_samples=1):
        """Update meter with new value `val`, the average of `num` samples.

        Args:
            val (float): Average value to update the meter with.
            num_samples (int): Number of samples that were averaged to
                produce `val`.
        """
        self.count += num_samples
        self.sum += val * num_samples
        self.avg = self.sum / self.count


def load_model(model, checkpoint_path, gpu_ids, return_step=True):
    """Load model parameters from disk.

    Args:
        model (torch.nn.DataParallel): Load parameters into this model.
        checkpoint_path (str): Path to checkpoint to load.
        gpu_ids (list): GPU IDs for DataParallel.
        return_step (bool): Also return the step at which checkpoint was saved.

    Returns:
        model (torch.nn.DataParallel): Model loaded from checkpoint.
        step (int): Step at which checkpoint was saved. Only if `return_step`.
    """
    device = f"cuda:{gpu_ids[0] if gpu_ids else 'cpu'}"
    ckpt_dict = torch.load(checkpoint_path, map_location=device)

    # Build model, load parameters
    model.load_state_dict(ckpt_dict['model_state'])

    if return_step:
        step = ckpt_dict['step']
        return model, step

    return model

def get_logger(log_dir, name):
    """Get a `logging.Logger` instance that prints to the console
    and an auxiliary file.

    Args:
        log_dir (str): Directory in which to create the log file.
        name (str): Name to identify the logs.

    Returns:
        logger (logging.Logger): Logger instance for logging events.
    """
    class StreamHandlerWithTQDM(logging.Handler):
        """Let `logging` print without breaking `tqdm` progress bars.

        See Also:
            > https://stackoverflow.com/questions/38543506
        """
        def emit(self, record):
            try:
                msg = self.format(record)
                tqdm.tqdm.write(msg)
                self.flush()
            except (KeyboardInterrupt, SystemExit):
                raise
            except:
                self.handleError(record)

    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    # Log everything (i.e., DEBUG level and above) to a file
    log_path = os.path.join(log_dir, 'log.txt')
    file_handler = logging.FileHandler(log_path)
    file_handler.setLevel(logging.DEBUG)

    # Log everything except DEBUG level (i.e., INFO level and above) to console
    console_handler = StreamHandlerWithTQDM()
    console_handler.setLevel(logging.INFO)

    # Create format for the logs
    file_formatter = logging.Formatter('[%(asctime)s] %(message)s',
                                       datefmt='%m.%d.%y %H:%M:%S')
    file_handler.setFormatter(file_formatter)
    console_formatter = logging.Formatter('[%(asctime)s] %(message)s',
                                          datefmt='%m.%d.%y %H:%M:%S')
    console_handler.setFormatter(console_formatter)

    # add the handlers to the logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

In [18]:
class BertFineTune(nn.Module):
    def __init__(self):
        super(BertFineTune, self).__init__()
        self.embed_model = BertModel.from_pretrained('bert-base-uncased')
        
        # initial Feed Forward Network with same size and tanh activation, and initial dropout
        self.dense = nn.Linear(768, 768)
        self.initial_activation = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)
        
        # stack more layer for better fitting and dropout
        self.fc_1 = nn.Linear(768, 256)
        self.tanh1 = nn.Tanh()
        self.dropout2 = nn.Dropout(p=0.1)

        # stack another layer
        self.fc_2 = nn.Linear(256, 128)
        self.tanh2 = nn.Tanh()

        # does classification
        self.classifier = nn.Linear(128, 5)

        # layers initialization
        nn.init.xavier_uniform_(self.dense.weight)  # layer initialization
        nn.init.xavier_uniform_(self.fc_1.weight)  # layer initialization
        nn.init.xavier_uniform_(self.fc_2.weight)  # layer initialization
        nn.init.xavier_uniform_(self.classifier.weight)  # layer initialization

        # If using BertForSequenceClassification
        # self.softmax_layer = nn.LogSoftmax(dim = 1)  # uses logsoftmax for NLL loss instead of normal softmax
        # self.embed_model_sequence = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 5, 
        #                                               output_attentions = False, output_hidden_states = False)


    def forward(self, x, seg_id_tensor, attnmask_tensor):
        # use bert to output embeddings
        # take the BERT embedding
        # bert_output[0] = last_layer_embedding  (batch_size, seq_len, hidden_size)
        # bert_output[1] = pooler_output, [CLS] embedding further preprocessed by linear and Tanh layers
        # bert_output[2] = tuple of length 13 (one for output of embedding layer and 12 output for each layer in the transformer) 
        bert_output = self.embed_model(x, token_type_ids=seg_id_tensor, attention_mask=attnmask_tensor)
        
        # sequence_output size = batch_size x sequence_length x hidden_size
        sequence_output = bert_output[0]

        # get embedding of [CLS] token
        cls_embed = sequence_output[:, 0, :] # take CLS embedding
        
        # post process the embedding layer by applying dense layer and tanh activation
        pooled_output = self.dense(cls_embed) # take linear layer
        pooled_output = self.initial_activation(pooled_output) # take activation

        # perform dropout and stack linear layers and tanh afterwards 
        after_dropout1 = self.dropout(pooled_output)
        
        linear1 = self.fc_1(after_dropout1)
        tanh1 = self.tanh1(linear1)
        after_dropout2 = self.dropout2(tanh1)
        linear2 = self.fc_2(after_dropout2)
        tanh2 = self.tanh2(linear2)

        # do classification
        logits = self.classifier(tanh2)
        
        # if using BertForSequenceClassificatioin
        # pooled_output = bert_output[1]
        # loss, logits = self.embed_model_sequence(x, 
        #                      token_type_ids=seg_id_tensor, 
        #                      attention_mask=attnmask_tensor, 
        #                      labels=labels)
                
        return logits

In [19]:
def get_prediction(review_ids, log_softmax_score):
    """
    review_ids (int): Tensor of Review example IDs.
    log_softmax_score (list): tensor of log likehood scores (take max to get prediction)
    """
    maxs = torch.argmax(log_softmax_score, dim = 1)

    pred_dict = {}
    for id, max_val in zip(review_ids, maxs):
        pred_dict[id.item()] = max_val.item() 
    return pred_dict

In [20]:
def evaluate_dict(gold_dict, pred_dict):
    sum_acc = 0
    sum_sentiment = 0
    total = 0
    all_preds = []
    all_gold = []
    
    for key, value in pred_dict.items():
        total += 1
        ground_truths = gold_dict[key]
        prediction = value
        if ground_truths == prediction:
            sum_acc += 1

        # get sentiment correct
        if (int(prediction) == 0 or int(prediction) == 1) and (int(ground_truths) == 0 or int(ground_truths) == 1):  # Negative
            sum_sentiment += 1
        elif (int(prediction) == 2) and (int(ground_truths) == 2): # Neutral
            sum_sentiment += 1
        elif (int(prediction) == 3 or int(prediction) == 4) and (int(ground_truths) == 3 or int(ground_truths) == 4): # Positive
            sum_sentiment += 1
        
        all_preds.append(int(prediction))
        all_gold.append(int(ground_truths))

    recallscore = recall_score(all_gold, all_preds, average='macro')
    precisionscore = precision_score(all_gold, all_preds, average = 'macro')
    f1score = f1_score(all_gold, all_preds, average='macro')

    eval_dict = {'acc': 100. * sum_acc / total,
                 'sentiment_acc': 100. * sum_sentiment / total,
                 'recall': 100. * recallscore,
                 'precision': 100. * precisionscore, 
                 'f1_score': 100. * f1score}

    return eval_dict

In [21]:
def evaluate(model, data_loader, eval_file, device):
    nll_meter = AverageMeter()

    model.eval()
    pred_dict = {}

    # get all true labels for ratings
    test_dataset = np.load(eval_file)
    true_labels = torch.from_numpy(test_dataset['rating']).long()
    uniq_ids = torch.from_numpy(test_dataset['ids']).long()
    true_labels = true_labels - 1 ## Need to subtract 1 because classes need to be 0 to n_classes - 1
    gold_dict = {}
    for true, ids in zip(true_labels, uniq_ids):
        gold_dict[ids.item()] = true.item()

    with torch.no_grad(), tqdm.notebook.tqdm(total=len(data_loader.dataset), position=1, leave=True) as progress_bar:
        for text, attnmask, seg_id, rating, ids  in data_loader:
            # Setup for forward
            text = text.to(device)
            attnmask = attnmask.to(device)
            seg_id = seg_id.to(device)
            batch_size = text.size(0)
            ids = ids.to(device)

            # rating needs to be 0 to num_classes - 1
            rating = rating - 1
            rating = rating.to(device)

            # Forward
            logits = model(text, seg_id, attnmask)
            loss_fct = nn.CrossEntropyLoss()  # nn.CrossEntropyLoss = nn.Log_SoftMax + nn.NLL_LOSS
            loss = loss_fct(logits.view(-1, 5), rating.view(-1))
            
            nll_meter.update(loss.item(), batch_size)

            # Get maximum prediction for prediction
            preds = get_prediction(ids, logits)
            pred_dict.update(preds)

            # Log info
            progress_bar.update(batch_size)
            progress_bar.set_postfix(NLL=nll_meter.avg)

    model.train()

    results = evaluate_dict(gold_dict, pred_dict)
    results_list = [('NLL', nll_meter.avg),
                    ('acc', results['acc']),
                    ('sentiment_acc', results['sentiment_acc']),
                    ('precision', results['precision']),
                    ('recall', results['recall']),
                    ('f1_score', results['f1_score'])
                    ]

    results = OrderedDict(results_list)
    return results, pred_dict

In [22]:
def visualize(tbx, pred_dict, eval_path, step, split, num_visuals):
    """Visualize text examples to TensorBoard.

    Args:
        tbx (tensorboardX.SummaryWriter): Summary writer.
        pred_dict (dict): dict of predictions of the form id -> pred.
        eval_path (str): Path to eval JSON file.
        step (int): Number of examples seen so far during training.
        split (str): Name of data split being visualized.
        num_visuals (int): Number of visuals to select at random from preds.
    """
    if num_visuals <= 0:
        return
    if num_visuals > len(pred_dict):
        num_visuals = len(pred_dict)

    visual_ids = np.random.choice(list(pred_dict), size=num_visuals, replace=False)

    with open(eval_path, 'r') as eval_file:
        eval_dict = json.load(eval_file)
    for i, id_ in enumerate(visual_ids):
        pred = pred_dict[id_] + 1 # NEED POST PROCESSING BECAUSE WE SUBTRACTED TO COMPLY WITH LOSS LABELS (0, n_classes - 1)
        example = eval_dict[str(id_)]
        user_id = example['user_id']
        business_id = example['business_id']
        text = example['text']
        gold = int(example['rating'])

        gold_sent = 'Negative' 
        pred_sent = 'Negative'

        if (int(pred) == 1 or int(pred) == 2):
            pred_sent = 'Negative'
        elif (int(pred) == 3):
            pred_sent = 'Neutral'
        elif (int(pred) == 4 or int(pred) == 5):
            pred_sent = 'Positive'
        
        if (int(gold) == 1 or int(gold) == 2):
            gold_sent = 'Negative'
        elif (int(gold) == 3):
            gold_sent = 'Neutral'
        elif (int(gold) == 4 or int(gold) == 5):
            gold_sent = 'Positive'


        tbl_fmt = (f'- **Reviews:** {text}\n'
                   + f'- **Answer:** {gold}\n'
                   + f'- **True Sentiment**: {gold_sent}\n'
                   + f'- **Prediction:** {pred}\n'
                   + f'- **Prediction Sentiment**: {pred_sent}')
        tbx.add_text(tag=f'{split}/{i+1}_of_{num_visuals}',
                     text_string=tbl_fmt,
                     global_step=step)

In [23]:
def get_available_devices():
    """Get IDs of all available GPUs.

    Returns:
        device (torch.device): Main device (GPU 0 or CPU).
        gpu_ids (list): List of IDs of all GPUs that are available.
    """
    gpu_ids = []
    if torch.cuda.is_available():
        gpu_ids += [gpu_id for gpu_id in range(torch.cuda.device_count())]
        device = torch.device(f'cuda:{gpu_ids[0]}')
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')

    return device, gpu_ids

In [28]:
save_dir = 'gdrive/My Drive/Yelp_Sentiment_Analysis/save/test/baseline-01'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

log = get_logger(save_dir, 'baseline')
tbx = SummaryWriter(save_dir)
log.info(f'Using random seed 224 ...')

device, gpu_ids = get_available_devices()
model = load_model(BertFineTune(), 'gdrive/My Drive/Yelp_Sentiment_Analysis/save/train/baseline-01/step_180000.pth.tar', gpu_ids, return_step=False)
model = model.to('cuda:0')
model.eval()

[08.04.20 16:56:41] Using random seed 224 ...
[08.04.20 16:56:41] Using random seed 224 ...
[08.04.20 16:56:41] Using random seed 224 ...


BertFineTune(
  (embed_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [31]:
epoch = 0
step = 0
device = 'cuda:0'
steps_till_eval = 0


with torch.no_grad(), tqdm.tqdm(total=len(dev_loader.dataset), position=0, leave=True) as progress_bar:
    for text, attnmask, seg_id, rating, ids in dev_loader:
        # Setup for forward
        text = text.to(device)
        attnmask = attnmask.to(device)
        seg_id = seg_id.to(device)
        rating = rating.to(device)
        batch_size = text.size(0)

        # need ratings to be 0 to n_classes - 1 (can later transform it)
        rating = rating - 1

        # Forward
        logits = model(text, seg_id, attnmask)

        # ypred
        ypred = torch.argmax(logits, dim = 1)

        # compute loss
        loss_fct = nn.CrossEntropyLoss()  # nn.CrossEntropyLoss = nn.Log_SoftMax + nn.NLL_LOSS
        loss = loss_fct(logits.view(-1, 5), rating.view(-1))
        loss_val = loss.item()

        # Log info
        step += batch_size
        progress_bar.update(batch_size)
        progress_bar.set_postfix(epoch=epoch, NLL=loss_val)
        
        tbx.add_scalar('train/NLL', loss_val, step)


        steps_till_eval -= batch_size
        if steps_till_eval <= 0:
            steps_till_eval = 10000

            # Evaluate and save checkpoint
            log.info(f'Evaluating at step {step}...')
            results, pred_dict = evaluate(model, dev_loader, 'gdrive/My Drive/Yelp_Sentiment_Analysis/test.npz', device)
            
            # Log to console
            results_str = ', '.join(f'{k}: {v:05.2f}' for k, v in results.items())
            log.info(f'Dev {results_str}')

            # Log to TensorBoard
            log.info('Visualizing in TensorBoard...')
            for k, v in results.items():
                  tbx.add_scalar(f'dev/{k}', v, step)

            visualize(tbx,
                      pred_dict=pred_dict,
                      eval_path='dataset/test.json',
                      step=step,
                      split='dev',
                      num_visuals=30)

  0%|          | 16/10000 [00:00<10:04, 16.52it/s, NLL=0.726, epoch=0]

[08.04.20 17:11:41] Evaluating at step 16...
[08.04.20 17:11:41] Evaluating at step 16...
[08.04.20 17:11:41] Evaluating at step 16...


  0%|          | 16/10000 [06:57<10:04, 16.52it/s, NLL=0.726, epoch=0]


[08.04.20 17:18:37] Dev NLL: 00.78, acc: 66.33, sentiment_acc: 84.16, precision: 66.13, recall: 66.32, f1_score: 66.19
[08.04.20 17:18:37] Dev NLL: 00.78, acc: 66.33, sentiment_acc: 84.16, precision: 66.13, recall: 66.32, f1_score: 66.19
[08.04.20 17:18:37] Dev NLL: 00.78, acc: 66.33, sentiment_acc: 84.16, precision: 66.13, recall: 66.32, f1_score: 66.19
[08.04.20 17:18:37] Visualizing in TensorBoard...
[08.04.20 17:18:37] Visualizing in TensorBoard...
[08.04.20 17:18:37] Visualizing in TensorBoard...


  2%|▏         | 224/10000 [07:07<5:10:53,  1.91s/it, NLL=0.953, epoch=0]


KeyboardInterrupt: ignored

In [ ]:
def get_available_devices():
    """Get IDs of all available GPUs.

    Returns:
        device (torch.device): Main device (GPU 0 or CPU).
        gpu_ids (list): List of IDs of all GPUs that are available.
    """
    gpu_ids = []
    if torch.cuda.is_available():
        gpu_ids += [gpu_id for gpu_id in range(torch.cuda.device_count())]
        device = torch.device(f'cuda:{gpu_ids[0]}')
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')

    return device, gpu_ids

In [ ]:
def main():
    device, gpu_ids = get_available_devices()
    model = load_model(model, 'gdrive/My Drive/Yelp_Sentiment_Analysis/save/train/baseline-01/step_180000.pth.tar', gpu_ids, return_step=False)
    model = model.to(device)
    model.eval()
    